# Rozpoznanie pojedyńczej cyfry na captchy

In [ ]:
import numpy as np
import keras
from keras import backend as K
from keras.preprocessing.image import img_to_array, load_img
from skimage.io import imread
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Wczytanie danych
Dane ułożone są w 10 folderach o nazwach od '0' do '9'. Przeiterujemy po folderach wczytując obrazki z każdego z nich zachowując nazwę folderu jako label.

In [ ]:
def load_data(path):
    # obrazki
    images = []
    # etykiety
    labels = []
    for i in range(10):
        # tworzymy ściężkę do folderu przez połączenie 
        # ścieżki do folderu z danymi z cyfra od 0 do 9
        folder_path = os.path.join(path, str(i)) 
        
        # pętla do iterowania po folderach
        for img in os.listdir(folder_path):
            
            # wczytanie obrazka
            image = imread(os.path.join(folder_path, img))
            
            # zamiana na numpy.ndarray
            x = img_to_array(image)
            
            # średnia zamiani nam kolorowy obrazek na czarnobiały
            # Można też skorzystać z funkcji dostępnej w open cv
            # która prawdopodobnie zamieni to trochę lepiej ;)
            x = np.mean(x, axis = 2)[:, :, np.newaxis]
            
            # dołączamy obrazek do przykładów treningowych
            images.append(x)
            labels.append(i)
            
    # shuffling
    indices = np.arange(len(labels))
    np.random.shuffle(indices)
            
    return np.array(images)[indices], np.array(labels)[indices]

In [ ]:
X, y = load_data('./dane/')

Wypluło nam 20'000 obrazków w rozmiarze 60x60 i z jednym kanałem

In [ ]:
X.shape

In [ ]:
# dzielimy dane na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
X_test.shape

In [ ]:
class our_scaler:
    '''
        w sklearn znajduje się StandardScaler służący do normalizacji danych.
        Z jakiegoś powodu nie działał w tym przypadku dlatego napisałem swój :))
        std = odchylenie standardowe
        mean = średnia
    '''
    def __init__(self):
        self.mean = 0
        self.std = 1
        
    def fit(self, data):
        self.mean = np.mean(data)
        self.std = np.std(data)
        
    def transform(self, data):
        return (data - self.mean) / self.std
    
    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)
    
    

In [ ]:
scaler = our_scaler()

# ważne jest aby normalizować obydwa zbiory tymi samymi wartościami
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# oczekujemy że odchylenia zbioru testowego będzie zbliżone do 1
np.std(X_test), np.mean(X_test)

In [ ]:
# wyplotujemy rozkład nasyceń pikseli losowego obrazka
plt.hist(X_train[10000].flatten())
plt.show()

Widzmy że rozkład jest bardzo nie normalny co będzie sprawiało problemy przy uczeniu sieci neuronowej.

In [ ]:
np.min(X_train), np.max(X_train)

Kodujemy etykiety do postaci one hot. Wektor one-hot jest n wymiarowym wektorem składającym się z samych zer i jednej jedynki. Długość wektora jest równa ilości różnych etykiet. 3 -> [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]

In [ ]:
one_hot_targets = np.eye(10)[y_train]

In [ ]:
print(y_train[:5])
print(one_hot_targets[:5])

### Sieć neuronowa
Zbudujemy konwolucyjną sieć neuronową. W klasycznej sieci neuronowej, na końcu, znajduje się szereg warstw w pełni połączonych.

Ten model będzie się prawdopodobnie długo uczył na procesorze (ok. 30 min) dlatego w komórce niżej wczytujemy juz przeuczony przeze mnie model ;)

#### Tworzenie modeli w kerasie
Jest na świecie mało rzeczy łatwiejszych od stworzenia sieci neuronowej w kerasie. Jest to biblioteka stworzona do bezwysiłkowego tworzenia prostych modeli. Do naszych potrzeb będziemy potrzebowali modelu sekwencyjnego Sequential który kieruje wyjście poprzedniej warstwy na wejście następnej. So modelu dodajemy wartswy metodą .add po koleji.

**keras.models.Sequential**
Model typu sequential, tak samo jak chyba każdy model w kerasie, ma 4 najważniejsze metody:
* compile - po dodaniu warstw, metoda rzeczywiście stowrzy nam model. Posiada następujące parametry:
    * optimizer - optymalizator np. keras.optimizers.Adam() lub SGD()
    * loss - zwykle string zawierający nazwę funkcji np. 'categorical_crossentropy' ale może też być funkcja
    * metrics - metryki którymi chcemy oceniać model np. ['accuracy']
* fit - dopasowuje model do danych. parametry:
    * x = dane wejściowe
    * y = labele
    * batch_size = wielkość batcha
    * epochs = ile razy chcemy pokazać cały dataset naszej sieci neuronowej
    * callbacks = bardzo ważne w realnej pracy - dla chętnych w dokumentacji ;)
* predict - przepuszcza dane przez sieć
    * x = nasze dane
    * ma też kilka innych, mniej ważnych parametrów
    

**keras.layers.Conv2D**
Conv2D jest warstwą konwolucyjną, gwiazdą wieczoru :)

parametry:
* filters - liczba filtrów w warstwie
* kernel_size - wielkość filtra np. (3, 3) lub (5, 5)
* strides - na razie nie ważne
* padding - na razie nie ważne

In [ ]:
# model sekwencyjny czyli wyjście z 
# warstwy poprzedniej jest kierowane do następnej
model = keras.models.Sequential()

# dodanie warstwy do modelu. W tym przypadku jest to warstwa konwolucyjna z 32 filtrami rozmiaru 5x5. 
# Ze względu na rozkład nasycenia pikseli funckją altywacji jest elu zamist relu. Zmniejsza to ilość martwych neuronów.
# input_shape jest wielkości (None, None, 1) ponieważ None dane nam mozliwość wrzucenia do sieci większego obrazka.
activation_function = 'elu'
model.add(keras.layers.Conv2D(32, kernel_size = (5, 5), activation = activation_function, input_shape = (60, 60, 1)))
model.add(keras.layers.Conv2D(64, kernel_size = (5, 5), activation = activation_function))
#model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(32, kernel_size = (3, 3), activation = activation_function))
model.add(keras.layers.MaxPool2D())
model.add(keras.layers.Conv2D(64, kernel_size = (5, 5), activation = activation_function))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Conv2D(64, kernel_size = (3, 3), activation = activation_function))

#model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Conv2D(64, kernel_size = (3, 3), activation = activation_function))
#model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.MaxPool2D())

# Global max pooling bierze macierz wektorów aktywacji po czym maxuje względem macierzy (wytłumaczę live :)) )
model.add(keras.layers.GlobalAveragePooling2D())

model.add(keras.layers.Dense(128, activation = activation_function))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(10))


model.add(keras.layers.Softmax())
#model.add(keras.layers.Lambda(lambda x: K.clip(x, 1e-4, 0.999)))

model.compile(keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-5), 
              'categorical_crossentropy',
             metrics = [keras.metrics.categorical_accuracy])

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, one_hot_targets, epochs = 8, batch_size = 64, shuffle = True, validation_split = 0.15, callbacks = [keras.callbacks.TerminateOnNaN()])

In [ ]:
#model.save('model_1_pretrained_bootcamp.h5')

In [ ]:

model_path = './model_1_pretrained_bootcamp.h5'
model = keras.models.load_model(model_path)

In [ ]:
np.set_printoptions(precision=3, suppress=True)

In [ ]:
for i in range(3):
    plt.imshow(np.squeeze(X_test[i]))
    preds = model.predict(X_test[i][np.newaxis, :, :, :])
    print('wyjście modelu: ', preds)
    print('argmax: \t', np.argmax(preds))
    print('GT: \t\t', y_test[i])
    plt.show()

In [ ]:
test_preds = model.predict(X_test)
test_preds = [np.argmax(t) for t in test_preds]

In [ ]:
test_preds[:5]

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(confusion_matrix(y_true = y_test, y_pred = test_preds))
plt.colorbar()
plt.title('Macierz pomyłek')
plt.show()

In [ ]:
plt.figure(figsize = (10, 10))
plt.imshow(confusion_matrix(y_true = y_test, y_pred = test_preds) *(1 - np.eye(10)))
plt.colorbar()
plt.title('Macierz pomyłek z wyzerowaną przekątną')